In [ ]:
!pip install -q transformers peft bitsandbytes datasets accelerate trl sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 19.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset
from trl import SFTTrainer
import os

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print("All libraries imported ")

CUDA Available: True
GPU: Tesla T4
GPU Memory: 15.6 GB
All libraries imported 


In [ ]:
r      = 16
lr     = 2e-4
batch  = 4
epochs = 3

print(f"r      = {r}")
print(f"lr     = {lr}")
print(f"batch  = {batch}")
print(f"epochs = {epochs}")
print("Hyperparameters set ")

r      = 16
lr     = 0.0002
batch  = 4
epochs = 3
Hyperparameters set 


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("4-bit config ready ")

4-bit config ready 


In [ ]:
MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

print("Model loaded in 4-bit ")
print("Gradient checkpointing enabled ")

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=r,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


In [ ]:
dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train")
dataset = dataset.select(range(500))

print(f"Dataset loaded ")
print(f"Total examples: {len(dataset)}")
print(f"Columns: {dataset.column_names}")

In [ ]:
def format_text(example):
    return {
        "text": f"Instruction:\n{example['instruction']}\n\n Input:\n{example['input']}\n\n Response:\n{example['output']}"
    }

dataset = dataset.map(format_text)

print("Dataset formatted ")
print(dataset[0]['text'])

In [ ]:
os.makedirs("/content/adapters", exist_ok=True)

training_args = TrainingArguments(
    output_dir="/content/adapters",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch,
    learning_rate=lr,
    bf16=True,
    gradient_checkpointing=True,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    report_to="none",
)

print("Training arguments set")

Training arguments set


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()
print("Training complete ")

In [ ]:
trainer.model.save_pretrained("/content/adapters")
tokenizer.save_pretrained("/content/adapters")

print("Adapter weights saved ")

Adapter weights saved 


In [16]:
import os

files = os.listdir("/content/adapters")
print("Files saved in /content/adapters:")
for f in files:
    print(f"   {f}")

Files saved in /content/adapters:
   adapter_model.safetensors
   checkpoint-250
   adapter_config.json
   README.md
   checkpoint-125
   tokenizer_config.json
   checkpoint-375
   chat_template.jinja
   tokenizer.json
